In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from nlstruct.dataloaders.medic import get_raw_medic

In [3]:
from __future__ import absolute_import
import argparse
import numpy as np
import torch
import os
import sys
import logging
import pdb

sys.path.insert(0,'/home/ytaille/AttentionSegmentation')

from allennlp.data import Vocabulary
from allennlp.data.iterators import DataIterator
# import allennlp.data.dataset_readers as Readers
import AttentionSegmentation.reader as Readers

# import model as Models
import AttentionSegmentation.model.classifiers as Models

from AttentionSegmentation.commons.utils import \
    setup_output_dir, read_from_config_file
from AttentionSegmentation.commons.model_utils import \
    construct_vocab, load_model_from_existing
# from AttentionSegmentation.visualization.visualize_attns import \
#     html_visualizer
import AttentionSegmentation.model.attn2labels as SegmentationModels

"""The main entry point

This is the main entry point for training HAN SOLO models.

Usage::

    ${PYTHONPATH} -m AttentionSegmentation/main
        --config_file ${CONFIG_FILE}

"""
args = type('MyClass', (object,), {'content':{}})()
args.config_file = 'Configs/config_ncbi_1class.json'
args.log = 'INFO'
args.loglevel = 'INFO'
args.seed = 1

# Setup Experiment Directory
config = read_from_config_file(args.config_file)
if args.seed > 0:
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if config.get('trainer', None) is not None and \
       config.get('trainer', None).get('cuda_device', -1) > 0:
        torch.cuda.manual_seed(args.seed)
serial_dir, config = setup_output_dir(config, args.loglevel)
logger = logging.getLogger(__name__)

# Load Training Data
TRAIN_PATH = config.pop("train_data_path")
logger.info("Loading Training Data from {0}".format(TRAIN_PATH))
dataset_reader_params = config.pop("dataset_reader")
reader_type = dataset_reader_params.pop("type", None)
assert reader_type is not None and hasattr(Readers, reader_type),\
    f"Cannot find reader {reader_type}"
reader = getattr(Readers, reader_type).from_params(dataset_reader_params)
instances_train = reader.read(file_path=TRAIN_PATH)
instances_train = instances_train
logger.info("Length of {0}: {1}".format(
    "Training Data", len(instances_train)))

# Load Validation Data
VAL_PATH = config.pop("validation_data_path")
logger.info("Loading Validation Data from {0}".format(VAL_PATH))
instances_val = reader.read(VAL_PATH)
instances_val = instances_val
logger.info("Length of {0}: {1}".format(
    "Validation Data", len(instances_val)))

# Load Test Data
TEST_PATH = config.pop("test_data_path", None)
instances_test = None
if TEST_PATH is not None:
    logger.info("Loading Test Data from {0}".format(TEST_PATH))
    instances_test = reader.read(TEST_PATH)
    instances_test = instances_test
    logger.info("Length of {0}: {1}".format(
        "Testing Data", len(instances_test)))

# # Load Pretrained Existing Model
# load_config = config.pop("load_from", None)

# # Construct Vocabulary
vocab_size = config.pop("max_vocab_size", -1)
logger.info("Constructing Vocab of size: {0}".format(vocab_size))
vocab_size = None if vocab_size == -1 else vocab_size
vocab = Vocabulary.from_instances(instances_train,
                                  max_vocab_size=vocab_size)
vocab_dir = os.path.join(serial_dir, "vocab")
assert os.path.exists(vocab_dir), "Couldn't find the vocab directory"
vocab.save_to_files(vocab_dir)

# if load_config is not None:
#     # modify the vocab from the source model vocab
#     src_vocab_path = load_config.pop("vocab_path", None)
#     if src_vocab_path is not None:
#         vocab = construct_vocab(src_vocab_path, vocab_dir)
#         # Delete the old vocab
#         for file in os.listdir(vocab_dir):
#             os.remove(os.path.join(vocab_dir, file))
#         # save the new vocab
#         vocab.save_to_files(vocab_dir)
logger.info("Saving vocab to {0}".format(vocab_dir))
logger.info("Vocab Construction Done")

# # Construct the data iterators
logger.info("Constructing Data Iterators")
data_iterator = DataIterator.from_params(config.pop("iterator"))
data_iterator.index_with(vocab)

logger.info("Data Iterators Done")

# Create the model
logger.info("Constructing The model")
model_params = config.pop("model")
model_type = model_params.pop("type")
assert model_type is not None and hasattr(Models, model_type),\
    f"Cannot find reader {model_type}"
model = getattr(Models, model_type).from_params(
    vocab=vocab,
    params=model_params,
    label_indexer=reader.get_label_indexer()
)
logger.info("Model Construction done")

# visualize = config.pop("visualize", False)
# visualizer = None
# if visualize:
#     visualizer = html_visualizer(vocab, reader)
segmenter_params = config.pop("segmentation")
segment_class = segmenter_params.pop("type")
segmenter = getattr(SegmentationModels, segment_class).from_params(
    vocab=vocab,
    reader=reader,
    params=segmenter_params
)

# logger.info("Segmenter Done")

# print("##################################\nAYYYYYYYYYYYYYYYYYYYYYYYY\n\n\n\n\n\n\n\n###########################")

# exit()


# if load_config is not None:
#     # Load the weights, as specified by the load_config
#     model_path = load_config.pop("model_path", None)
#     layers = load_config.pop("layers", None)
#     load_config.assert_empty("Load Config")
#     assert model_path is not None,\
#         "You need to specify model path to load from"
#     model = load_model_from_existing(model_path, model, layers)
#     logger.info("Pretrained weights loaded")

# logger.info("Starting the training process")



/home/ytaille/.conda/envs/deep_multilingual_normalization/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


2021-03-01 15:46:09,614: INFO: train_data_path = /home/ytaille/data/resources/medic/ncbi_conll_ner_1class_train.conll
2021-03-01 15:46:09,618: INFO: Loading Training Data from /home/ytaille/data/resources/medic/ncbi_conll_ner_1class_train.conll
2021-03-01 15:46:09,621: INFO: dataset_reader.type = WeakConll2003DatasetReader
2021-03-01 15:46:09,624: INFO: dataset_reader.token_indexers.bert.type = bert-pretrained
2021-03-01 15:46:09,626: INFO: dataset_reader.token_indexers.bert.pretrained_model = ./Data/embeddings/bert-base-multilingual-cased-vocab.txt
2021-03-01 15:46:09,627: INFO: dataset_reader.token_indexers.bert.use_starting_offsets = True
2021-03-01 15:46:09,628: INFO: dataset_reader.token_indexers.bert.do_lowercase = False
2021-03-01 15:46:09,629: INFO: dataset_reader.token_indexers.bert.never_lowercase = None
2021-03-01 15:46:09,631: INFO: dataset_reader.token_indexers.bert.max_pieces = 512
2021-03-01 15:46:09,635: INFO: loading vocabulary file ./Data/embeddings/bert-base-multilin

0it [00:00, ?it/s]

2021-03-01 15:46:09,789: INFO: Reading instances from lines in file at: /home/ytaille/data/resources/medic/ncbi_conll_ner_1class_train.conll


1803it [00:00, 2344.82it/s]

...

2021-03-01 15:46:10,892: INFO: Length of Testing Data: 316
2021-03-01 15:46:10,893: INFO: max_vocab_size = -1
2021-03-01 15:46:10,894: INFO: Constructing Vocab of size: -1
2021-03-01 15:46:10,895: INFO: Fitting token dictionary from dataset.



100%|██████████| 1803/1803 [00:00<00:00, 37386.93it/s]

2021-03-01 15:46:10,958: INFO: Saving vocab to ./trained_models/NCBI-BERT-realFT-PS/run-150/vocab
2021-03-01 15:46:10,959: INFO: Vocab Construction Done
2021-03-01 15:46:10,961: INFO: Constructing Data Iterators
2021-03-01 15:46:10,962: INFO: iterator.type = bucket
2021-03-01 15:46:10,963: INFO: iterator.sorting_keys = [['tokens', 'bert']]
2021-03-01 15:46:10,964: INFO: iterator.padding_noise = 0.1
2021-03-01 15:46:10,965: INFO: iterator.biggest_batch_first = False
2021-03-01 15:46:10,966: INFO: iterator.batch_size = 32
2021-03-01 15:46:10,967: INFO: iterator.instances_per_epoch = None
2021-03-01 15:46:10,967: INFO: iterator.max_instances_in_memory = None
2021-03-01 15:46:10,968: INFO: Data Iterators Done
2021-03-01 15:46:10,969: INFO: Constructing The model
2021-03-01 15:46:10,970: INFO: model.type = MultiClassifier
2021-03-01 15:46:10,971: INFO: model.method = binary
2021-03-01 15:46:10,972: INFO: model.text_field_embedder.type = basic
2021-03-01 15:46:10,973: INFO: model.text_field_

2021-03-01 15:46:16,093: INFO: Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}

2021-03-01 15:46:24,445: INFO: Layer encoder.layer.10.attention.self.query.weight is finetuned
2021-03-01 15:46:24,446: INFO: Layer encoder.layer.10.attention.self.query.bias is finetuned
2021-03-01 15:46:24,447: INFO: Layer encoder.layer.10.attention.self.key.weight is finetuned
2021-03-01 15:46:24,448: INFO: Layer encoder.layer.10.attention.self.key.bias is finetuned
2021-03-01 15:46:24,449: INFO: Layer encoder.layer.10.attention.self.value.wei

/home/ytaille/.conda/envs/deep_multilingual_normalization/lib/python3.6/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [4]:
# Necessary to add unknown tag to dictionnary to avoid errors later
data_iterator.vocab.add_token_to_namespace("@@UNKNOWN@@", "chunk_tags")

1141

In [5]:
config = read_from_config_file(args.config_file)


In [6]:
from AttentionSegmentation.trainer import Trainer

from nlstruct.utils import  torch_global as tg

trainer = Trainer.from_params(
    model=model,
    base_dir=serial_dir,
    iterator=data_iterator,
    train_data=instances_train,
    validation_data=instances_val,
    segmenter=segmenter,
    params=config.pop("trainer")
)


2021-03-01 15:46:24,887: INFO: PyTorch version 1.5.1 available.
2021-03-01 15:46:28,254: INFO: TensorFlow version 2.3.1 available.
2021-03-01 15:46:28,744: INFO: Loading faiss with AVX2 support.
2021-03-01 15:46:28,747: INFO: Loading faiss.
2021-03-01 15:46:29,191: INFO: trainer.patience = 10
2021-03-01 15:46:29,192: INFO: trainer.validation_metric = +accuracy
2021-03-01 15:46:29,193: INFO: trainer.num_epochs = 50
2021-03-01 15:46:29,194: INFO: trainer.cuda_device = 0
2021-03-01 15:46:29,195: INFO: trainer.grad_norm = None
2021-03-01 15:46:29,196: INFO: trainer.grad_clipping = None
2021-03-01 15:46:29,197: INFO: trainer.num_serialized_models_to_keep = 1
2021-03-01 15:46:31,784: INFO: trainer.optimizer.type = adam
2021-03-01 15:46:31,786: INFO: trainer.optimizer.parameter_groups = [[['.*bert.*'], ConfigTree([('lr', 2e-07)])], [['.*encoder_word.*', '.*attn.*', '.*logit.*'], ConfigTree([('lr', 0.001)])]]
2021-03-01 15:46:31,787: INFO: Converting Params object to dict; logging of default v

In [7]:
attn = torch.Tensor([[0,1,0,1,0], [0,0,1,0,0]])

In [8]:
attn_boosted = attn.clone()
nnz = (attn>0).nonzero().t().chunk(chunks=2,dim=0)

print(nnz)

new_nnz = [[], []]

for nz0, nz1 in zip(nnz[0][0].numpy(), nnz[1][0].numpy()):
    new_nnz[0].extend([nz0,nz0])
    new_nnz[1].extend([nz1-1,nz1+1])
    
new_nnz[0] = torch.Tensor([new_nnz[0]]).long()
new_nnz[1] = torch.Tensor([new_nnz[1]]).long()
new_nnz = (new_nnz[0], new_nnz[1])
print(new_nnz)
attn_boosted[new_nnz] += 0.1

attn_boosted

(tensor([[0, 0, 1]]), tensor([[1, 3, 2]]))
(tensor([[0, 0, 0, 0, 1, 1]]), tensor([[0, 2, 2, 4, 1, 3]]))


tensor([[0.1000, 1.0000, 0.1000, 1.0000, 0.1000],
        [0.0000, 0.1000, 1.0000, 0.1000, 0.0000]])

In [9]:

# USE BIO BERT
# TRAIN STEP 1 ONLY ON MEDIC LABELS (+ NCBI MENTIONS)
# PREPROCESS / TRAIN / ATTEINDRE BONS SCORES
# GET MEDIC ALTERNATIVE LABELS DANS NLSTRUCT -> TRADUIRE LABELS NCBI VERS MEDIC

# USE ENTROPY INSTEAD OF CROSS ENTROPY -> not rely on labelled data only (rely on model certainty)

# GROUPS : TYPE SEMANTIQUE À LA MENTION (pas utiliser)

# NGRAMS FOR ENTITIES -> not possible with discontinued entities

# Use "separation token" in phrases ?

# Use a limited number of attention heads (not one per class)

# Use same method as Perceval for trajectories (draw closest ones, reduce list, repeat) -> prédiction itérative

# Maybe remove weakly supervised completely?

# Test with Reinforce only after a few epochs

# Facteur de représentation pour pondérer loss de Perceval ?

# Plusieurs facteurs pour constituer la reward

# Facteur de similarité mention extraite / synonyme plutôt que similarité mention / label ?

# Make sure that every trajectory is different -> draw first then use Perceval

# Métrique finale : Est-ce qu'on arrive à choper les CUI ? -> parce que frontières entités dures à déterminer 

# Use only one class ? -> simpler because all mentions are diseases -> MAKE SURE THAT SEVERAL MENTIONS ARE PREDICTABLE

# maybe problem with reinforce comes from hyperparameters??

In [17]:
ddddddddddddddddddddddddddddddqsssssssssssssssssxw<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<





$pppppppppoooooooo







cxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxbffffffffffffffffqeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee

In [18]:
trainer.train()

2021-03-01 16:12:27,685: INFO: Beginning training.
2021-03-01 16:12:27,687: INFO: ==================================================
2021-03-01 16:12:27,688: INFO: Starting Training Epoch 1/50
2021-03-01 16:12:27,689: INFO: Peak CPU memory usage MB: 6866.448
2021-03-01 16:12:27,785: INFO: GPU 0 memory usage MB: 13726


/home/ytaille/AttentionSegmentation/allennlp/data/fields/array_field.py:42: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return_array[slices] = self.array
Disease: 1.0000, accuracy: 1.0000, loss: 0.7777 ||: 100%|██████████| 57/57 [11:44<00:00, 12.36s/it]

2021-03-01 16:24:13,217: INFO: Starting with Validation



Disease: 0.9718, accuracy: 0.9718, loss: 0.3557 ||: 100%|██████████| 10/10 [00:01<00:00,  7.65it/s]

2021-03-01 16:24:14,530: INFO: Validation done. (60.0 / 319) zero predicted


...

2021-03-01 17:00:33,864: INFO: Best validation performance so far. Copying weights to './trained_models/NCBI-BERT-realFT-PS/run-150/models/best.th'.
2021-03-01 17:00:40,879: INFO: Metrics:
                        Training accuracy          : 0.998  Validation accuracy          : 0.972
                        Training Disease           : 0.998  Validation Disease           : 0.972
                        Training loss              : 0.773  Validation loss              : 0.356

2021-03-01 17:00:40,880: INFO: Writing validation visualization at ./trained_models/NCBI-BERT-realFT-PS/run-150/visualization/validation.html


100%|██████████| 10/10 [00:02<00:00,  3.95it/s]


2021-03-01 17:00:43,720: INFO: Tag: Disease, Acc: 97.18
2021-03-01 17:00:43,722: INFO: Average ACC: 97.18
2021-03-01 17:00:43,776: INFO: processed 22501 tokens with 531 phrases; 
2021-03-01 17:00:43,777: INFO: found: 756 phrases; correct: 303.

2021-03-01 17:00:43,778: INFO: accuracy:  95.63%; 
2021-03-01 17:00:43,779: INFO: precision:  40.08%; recall:  57.06%; FB1:  47.09
2021-03-01 17:00:43,780: INFO:               TAG  precision   recall      FB1
2021-03-01 17:00:43,783: INFO:           Disease     40.08%   57.06%   47.09%
2021-03-01 17:00:43,785: INFO: Writing predictions to ./trained_models/NCBI-BERT-realFT-PS/run-150/predictions.json
2021-03-01 17:00:43,786: INFO: Reducing LR: 1.25e-08 -> 1.25e-08
2021-03-01 17:00:43,787: INFO: Epoch duration: 00:11:54
2021-03-01 17:00:43,788: INFO: Estimated training time remaining: 09:15:05
2021-03-01 17:00:43,788: INFO: ==================================================
2021-03-01 17:00:43,789: INFO: ===========================================

/home/ytaille/.conda/envs/deep_multilingual_normalization/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:603: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/home/ytaille/AttentionSegmentation/allennlp/data/fields/array_field.py:42: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return_array[slices] = self.array
  0%|          | 0/57

KeyboardInterrupt: 

In [ ]:
logger.info("Training Done.")
if instances_test is not None:
    logger.info("Computing final Test Accuracy")
    trainer.test(instances_test)
logger.info("Done.")